# Abstract / Introduction

The aim of this project is to review different methods of Question Answering in the machine learning literature by applying them to a dataset that was made availab from the Kaggle Challenge *TensorFlow 2.0 Question Answering*

It's not Google Knowledge Graph.

Massive collection of full-text documents today. It used to be enough to just search and send the list of documents containinf the answer but nowadays there are billions of files and thousands probably contain your answer. Can we rather send the answer to the question? Especially for mobile phone use.

Two parts: 

1. Still do information retreivel techniques, to find out which documents contain the answers. Traditional techniques are scalable over billions of documents. Whereas, current systems are not. But once we got our candidate, we'd like to search for the answer.

2. Findding the answer in the document: reading comprehensions

# Prior related work

## Reading comprehension

Started back in the 70s.

A brief history of Reading Comprehension:

- Schank, Abelson, Lehnert et al. c. 1977 - "Yale AI Project", nothing much came out of that.

- Revived by Lynette Hirschman in 1999: could NLP systems answer human reading comprehension questions from 3rd to 6th graders? Using simple methods.


- Revived again by Chris Burges in 2013 with MCTest: answering questions over simple stroy texts

Burges : "A machine comprehends a passage of text if, for any question regarding the text thatcan be answered correctly by a majority of native speakers, that machine can provide a string which those speakers would agree both answers that question, and does not contain information irrelevant to that question." from Towards the Machine Comprehension of Text: An Essay, December 23, 2013.

- Floodgates opened in 2015/16 with the production of *large datasets* which permit supervised neural systems to be built: Hemann et al. (NIPS 2015), Rajpurkar et al. (EMNLP 2016 - SQuAD), MS MARCO, TriviaQA, RACE, NewsQA, NarrativeQA, ...

## Open-domain Question Answering

- Simmons et al. (1964) did first exploration of answering questions from an expository text based on matching dependancy parses of a question and answer

- Murax (Kupiec 1993) aimed to answer questions over an online encyclopedia using IR and shallow linguistic processing

- The NIST TREC QA track begun in 1999 first rigourously investigated answering fact questions over a large collection of documents

- IBM's Jeopardy System (DeepQA, 2011) brought attention to a version of the problem; it used an ensemble of many methods

- DrQA (Chen et al. 2016) uses IR followed by neural reading comprehension to bring deep learning to Open-domain QA.

## Turn-of-the Millenium Full NLP QA:

- Complex systems but the did work fairly well on "factoid" questions, e.g. architecture of LCC (Harabagio/Moldovan) QA system, circa 2033

Factoid QA, people in NLP used it. e.g. What year was Elvis Presley born, what is the name of Beyonce's husband, etc. Anything for which there's a clear answer. Like most of web searches, these systems actually did work well in the past, up to 70%. However, it stayed there. These were very complex systems, with a huge amount of hand-built stuff. 

Contrast between this and Deep Learning.

## Stanford Question Answering Dataset (SQuAD)

By construction for SQuAD an answer is always a sub-sequence of words in a passage (a span). 

100k examples. Answer must be a span in the passage. aka extractive question answering

### SQuAD v1.1.

How it was built: They used Mechanical Turk to find three people to read and give answers--> three answers per question.

Examples, given a paragraph:

- Along with non-governmental and non-state schools, what is anbother name for schools? A1: indepedent, A2: independent schools, A3: independent schools

- Along with sport and art, what is a type of talent scholarship? A1: academic, A2: academic, A3: academic

Evaluation metrics:

1. Exact match (1/0)
2. F1: Take system and each gold answer as bag of words, evaluate macro-average of per-question F1 scores.

F1 explanation: you treat the system span on each gold answer as a bag of worlds.

P = the percent of words in the system's answer that are actually in a gold span
R = the percent of words in a gold span that are in the system's span

Calculate the harmonic mean of those two numbers, harmonic mean being a very conservative average so it's close to MIN of those two numbers. For each question then the F1 is the maximum F1 score over three different answers, then average the F1 of questions.

Why not use exact match? 

Even if it gives a little bit of robustness, three people is not a large sample. So statistically not meaningful.

Both metrics ignore punctuation and "a" and "the".

Humans had F1=91.2

Logistic regression baseline, F1 = 51.0.

BERT single = 91.835 - Google AI Language
BERT ensemble = 93.160
nlnet ensemble = 91.202 - Microsoft Research Asia
QANet ensemble = 90.490 - Google Brain and CMU
r-net ensemble = 90.147 - Microsoft Research Asis

### SQuAD 2.0

- A defect of SQuAB 1.0 is that all questions have an answer in the paragraph

- Systems (implicitly) rank candidates and choose the best one

- You don't have to judge whether a span answers the question

- In SQuAD 2.0, 1/3 of the training questions have no answer, and about 1/2 of the dev/test questions have no answer. For NoAnswer examples, NoAnswer receives a score of 1, and any other response gets 0, for both exact match and F1

- Simplest system approach to SQuAD 2.0: Have a threshold score for whether a span answers a question

- Or you could have a second component that confirms answering such as Natural Language Inference (NLI) or "Answer validation"

**SQuAD limitations:**

- SQuAD has a number of other key limitations too: (i) only span-based answers (yes/no, counting, implicit why), (ii) questions were constructed looking at the passages, and (iii) barely any multi-fact/sentence inference beyond coreference

Nevertheless, it is well-targeted, well-structured, clean dataset

- it thas been the most used and competed on QA dataset
- it has also been a useful starting point for building systems in industry (though in-domain data always really helps!)

# Model 0: Logistic Regression

# Model 1: Stanford Attentive Reader

"Talk about what models you've built for awhile."

Use **Stanford Attentive Reader** as your baseline.

How does it work?

The way it works is; first we have a question like "Which team won SuperBowl L?"

1. Buil a representation of a question as a vector.
2. For each word in question, we look up a word embedding (binary)
3. Then run an LSTM forward and backward. 
4. Grab the end state of LSTMs and simply concatenate them together into a vector of dimension 2D if our hidden states of LSTM are D and we say that is the representation of the question.
5. Then start looking at the passage. For the start of dealin with the passage, we do the same thing. That is, we look up a word vector for every word in the passage and we run a bidirectional LSTM, now being represented a bit more compactly across the passage.
6. Use the question representation to work out where the answer is using attention. This is a different use of attention to machine translation. The kind of attention equations are still exactly the same. But we've now got this one question vector that we're going to be trying to match against to return the answer. Work out an attention score between each word's bi-LSTM representation and the question. We're using the following bi-linear attention 

$$\alpha_{i} = \text{softmax}_{i}(\mathbf{q}^\top\mathbf{W}_{s}\tilde{\mathbf{p}}_i) $$

where $\mathbf{q}$ is the question vector, $ \tilde{\mathbf{p}} $ is the vector for a particular position in the passage, the two concatenated LSTM hidden states, $ \mathbf{W} $ is the learning matrix. We calculate that quantity for each position and then we put through a softmax which will give us probability over the different words in the passage. And those give us attention weights. So at that point, we have attention weights for different positions in the passafe and we just declare that that is where the answer starts.

7. To get the end of the answer, we do exactly the same thing again apart from we train a different $ \mathbf{W} $ matrix.

What if the most important words are in the middle of the answer? It's the bi-LSTM's job to push the limits to the extremes of the span, so that this simple bi-linear attention will get a big score at the start of the span. 

How come we use exactly the same equation to detect the beginning and the end of the answer? How come one of them is meant to know it's picking up beginning and the other the end? We just tell the neural network to learn a matrix for the beginning and a different one for the end. That constraint, will pressure the neural network to self orgnize itself in such a way that there will be some parts of this hidden representation that will be good at learning starts of spans, and some parts that will where the span's end is. The $ \mathbf{W} $ matrix will be able to pick out those parts of the representation.  

This very simple model works very well.

# Model 2: Stanford Attentive Reader++


https://github.com/peterkim95/attentive-reader



# Model 3: BERT?

http://mccormickml.com/

# Data

Professor in the video mention data after the models.\
"Discussion about what data you're using for your projects. "

# Experiments

"Figures and tables"

# Results

"More tables and figures". Results showing how your system works.

# Analysis and conclusion

"Error analysis to see what you got right and wrong"
"Plans for the future and conclusions"